<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Кубические сплайны

### Modules - Curve Fitting
<section class="post-meta">
By Jonas Tjemsland, Eilif S. Øyre and Jon Andreas Støvneng
</section>
Last edited: January 26th 2018 
___


*Сплайны* - это тип *интерполяционных* данных, метод (повторного)построения функции между заданным набором точек данных. Интерполяция может использоваться для представления сложных в вычислительном отношении функций, таких как, например, многочлены. Затем, используя таблицу из нескольких оценок функций, можно легко аппроксимировать истинную функцию с высокой точностью.

При сплайновой интерполяции данные интерполируются несколькими полиномами малой степени. Это отличается от полиномиальной интерполяции, при которой данные интерполируются одним полиномом высокого порядка. Для общего обсуждения полиномиальной интерполяции мы отсылаем вас к блокноту по [полиномиальной интерполяции](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/polynomial_interpolation.ipynb). Простейшим примером интерполяции сплайнов являются линейные сплайны, где точки данных просто соединены прямыми линиями. Мы обсудим интерполяцию *кубическими сплайнами*, которая интерполирует многочлены с использованием кубических многочленов с непрерывными первой и второй производными. Мы создадим алгоритм и некоторые функции для вычисления кубического сплайна. 

Начинаем с импорта необходимых пакетов и установки общих параметров рисунка.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.linalg as la

%matplotlib inline

In [ ]:
# Set some figure parameters
newparams = {'figure.figsize': (15, 7), 'axes.grid': False,
             'lines.markersize': 10, 'lines.linewidth': 2,
             'font.size': 15, 'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral'}
plt.rcParams.update(newparams)

## Простой Пример
Предположим, что нам даны четыре точки данных: $\{(0, 0), (1, -1), (2, 2), (3, 0)\}$. Кубический сплайн, интерполирующий эти точки,
$$
S(x) =
\begin{cases}
-\frac{12}{5}x + \frac{7}{5}x^3, & 0\leq x < 1,\\
-1 + \frac{9}{5}(x - 1) + \frac{21}{5}(x-1)^2 - 3(x-1)^3, & 1 \leq x < 2,\\
2 + \frac{6}{5}(x - 2) -\frac{24}{5}(x-2)^2 + \frac{8}{5}(x-2)^3, & 2 \leq x < 3.\\
\end{cases}
$$
Давайте построим точки данных, линейный сплайн и кубический сплайн!

In [ ]:
n = 200
x1 = np.linspace(0, 1, n)
x2 = np.linspace(1, 2, n)
x3 = np.linspace(2, 3, n)

# Cubic spline
S1 = -12/5*x1 + 7/5*x1**3
S2 = -1 + 9/5*(x2 - 1) + 21/5*(x2 - 1)**2 - 3*(x2 - 1)**3
S3 = 2 + 6/5*(x3 - 2) - 24/5*(x3 - 2)**2 + 8/5*(x3 - 2)**3
plt.plot(np.concatenate([x1, x2, x3]),  np.concatenate([S1, S2, S3]), label="Cubic spline")
# Linear spline
plt.plot([0, 1, 2, 3], [0, -1, 2, 0], "--", label="Linear spline")
# Data points
plt.plot([0, 1, 2, 3], [0, -1, 2, 0], "o", label="Data points")

plt.legend()
plt.show()

**Упражнение:** Убедитесь, что кубический сплайн выше имеет непрерывные первую и вторую производные.

## Определение

Общий кубический сплайн $S(x)$, интерполирующий $n$ точек данных $\{(x_1, y_1), (x_2, y_2),..., (x_n, y_n)\}$ можно записать в виде

\begin{equation}
S(x) =
\begin{cases}
S_1(x) &= y_1 + b_1(x - x_1) + c_1(x-x_1)^2 + d_1(x-x_1)^3, & \text{for } x\in[x_1, x_2],\\
S_2(x) &= y_2 + b_2(x - x_2) + c_2(x-x_2)^2 + d_2(x-x_2)^3, & \text{for } x\in[x_2, x_3],\\
&\vdots&\\
S_{n-1}
(x) &= y_{n-1} + b_{n-1}(x - x_{n-1}) + c_{n-1}(x-x_{n-1})^2 + d_{n-1}(x-x_{n-1})^3, & \text{for } x\in[x_{n-1}, x_n],\\
\end{cases}
\label{eq:spline}
\end{equation}

для некоторых констант $b_i, c_i, d_i$, $i=1, ..., n$. Как уже упоминалось во введении, мы требуем, чтобы сплайн был непрерывным и имел непрерывные первую и вторую производные. Это дает следующие свойства: [1]

&emsp;&emsp; 1\. $S_i(x_i)=y_i$ and $S_i(x_{i+1})=y_{i+1}$ for $i=1,...,n-1$,  
&emsp;&emsp; 2\. $S_{i-1}'(x_i)=S_{i}'(x_i)$ and $S_i(x_{i+1})=y_{i+1}$ for $i=2,...,n-1$,  
&emsp;&emsp; 3\. $S_{i-1}''(x_i)=S_{i}''(x_i)$ and $S_i(x_{i+1})=y_{i+1}$ for $i=2,...,n-1$.  

Эти три свойства гарантируют, что сплайн будет непрерывным и гладким.

Общее количество констант $b_i, c_i, d_i$, которые нам нужно вычислить, равно $3(n-1)$.

## Условия конечной точки

Обратите внимание, что общее количество условий, налагаемых указанными выше свойствами, составляет $3n-5$. Однако общее количество коэффициентов $b_i, c_i, d_i$, которые нам нужно вычислить, составляет $3(n-1)$. Следовательно, нам нужны два дополнительных условия, чтобы сделать сплайн $S(x)$ уникальным. Это достигается с помощью условий конечной точки.

Существует несколько вариантов условий конечной точки (см., например, [1]). Мы будем рассматривать *естественные кубические сплайны*,

&emsp;&emsp; 4a\. $S''_1(x_1)= 0$ и $S''_{n-1}(x_n)=0$,

и *кубические сплайны без узлов*

&emsp;&emsp; 4b\. $S_1'''(x_2)=S_2'''(x_2), \; S_{n-2}'''(x_{n-1})=S_{n-1}'''(x_{n-1})$.

&nbsp;&nbsp;&nbsp;&nbsp;**Упражнение:** Какое условие конечной точки используется в приведенном выше примере?

## Алгоритм

Из свойств 1, 2 и 3 получаем

\begin{equation}
y_{i+1} = y_{i}+b_{i}(x_{i+1}-x_i) + c_i(x_{i+1}-x_i)^2 + d_i(x_{i+1}-x_i)^3, \quad i=1,...,n-1,
\label{eq:prop1} \quad (1)
\end{equation}

\begin{equation}
0 = b_i + 2c_i(x_{i+1}-x_i) + 3d_i(x_{i+1}-x_i)^2-b_{i+1}, \quad i=1,...,n-2,
\label{eq:prop2}\quad (2)
\end{equation}

и

\begin{equation}
0 = c_i+3d_i(x_{i+1}-x_i)-c_{i+1}, \quad i=1,...,n-2,
\label{eq:prop3}\quad (3)
\end{equation}

соответственно. Вывод прямолинейен и оставлен в качестве упражнения для читателя. Если мы решим эти уравнения, мы получим константы $b_i, c_i, d_i$ и, следовательно, кубический сплайн. Чтобы упростить обозначение, мы определяем $\Delta x_i=x_{i+1}-x_i$ и $\Delta y_i = y_{i+1}-y_i$. Используя уравнения показанные выше, мы получаем следующие выражения для $b_i$ и $d_i$ в терминах коэффициентов $c$:

\begin{align}
d_i &= \frac{c_{i+1}-c_i}{3\Delta x_i}, \label{eq:d}\\
b_i &= \frac{\Delta y_i}{\Delta x_i}-\frac{1}{3}\Delta x_i (2 c_i + c_{i+1}).\label{eq:b}
\end{align}

И после подстановки получим,

$$\Delta x_ic_i + 2(\Delta x_i + \Delta x_{i+1})c_{i+1}+\Delta x_{i+2}c_{i+2} = 3\left(\frac{\Delta y_{i+1}}{\Delta x_{i+1}}-\frac{\Delta y_{i}}{\Delta x_{i}}\right)$$

что составляет $n-2$ уравнения для $c_1,..., c_n$. Условие конечной точки *естественного сплайна* дает $c_1=c_n=0$. Мы можем записать это в виде матричного уравнения

$$
\begin{pmatrix}
1 & 0 & 0 & 0 & \cdots & 0 & 0 \\
\Delta x_1 & 2\Delta x_1 + 2\Delta x_2 & \Delta x_2 & 0 & \cdots&0&0\\
0 & \Delta x_2 & 2\Delta x_2 + 2\Delta x_3 & \Delta x_3 & \cdots&0&0\\
\vdots & \vdots & \ddots & \ddots & \ddots & \vdots & \vdots\\
0 & 0 & 0 & 0 & \Delta x_{n-2} & 2\Delta x_{n-2} + 2\Delta x_{n-1} & \Delta x_{n-1}\\
0 & 0 & 0 & 0 & 0 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
c_1 \\ c_2 \\ \\ \vdots \\ \\ c_n
\end{pmatrix} = 
\begin{pmatrix}
0 \\
3\left(\frac{\Delta y_{2}}{\Delta x_2}-\frac{\Delta y_1}{\Delta x_1}\right) \\
\vdots
\\
3\left(\frac{\Delta y_{n-1}}{\Delta x_{n-1}}-\frac{\Delta y_{n-2}}{\Delta x_{n-2}}\right) \\
0
\end{pmatrix}.
$$

Алгоритм нахождения сплайна теперь совершенно очевиден. Мы начинаем с построения матричного уравнения, затем решаем его, чтобы найти $c_i$, и, в свою очередь, вычисляем $b_i$ и $d_i$.

Первая и последняя строки матрицы изменяются с помощью условий конечной точки *not-a-knot*. Обратите внимание, что свойство 4b подразумевает $d_1=d_2$ и $d_{n-2}=d_{n-1}$. Если мы подставим уго в уравнение для $d_i$, мы получим
$$\Delta x_2 c_1 -(\Delta x_1 + \Delta x_2) c_2 + \Delta x_1 c_3 = 0,$$
$$\Delta x_{n-1} c_{n-1} -(\Delta x_{n-2} + \Delta x_{n-1})c_{n-1}+\Delta x_{n-2}c_n = 0.$$
Первая строка с конечными условиями *not-a-knot* становится $$(\Delta x_2\;\; -(\Delta x_1 + \Delta x_2)\;\; \Delta x_1\;\; 0\;\; 0\;\; ...).$$ Аналогично, последняя строка становится $$(0\;\; ... \;\; 0 \;\; \Delta x_{n-1}\;\; -(\Delta x_{n-2} + \Delta x_{n-2})\;\; \Delta x_{n-2}).$$

Теперь мы перейдем к созданию функции, которая вычисляет кубический сплайн, интерполирующий точки $\{(x_1, y_1), (x_2, y_2),..., (x_n, y_n)\}$. Обратите внимание, что приведенное выше матричное уравнение является трехдиагональным, и поэтому его можно сохранить в виде массива $3\times n$ и эффективно решить с помощью [scipy.linalg.solve_banded](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve_banded.html). В случае *not-a-knot* матрица становится полосчатой матрицей с двумя верхними и двумя нижними диагоналями.

&nbsp;&nbsp;&nbsp;&nbsp;**Упражнение:** Выведите уравнения (1-3) из свойств 1-3.

In [ ]:
def cubic_spline_coeffs(x, y, endpoint="natural"):
    """ Вычисляет коэффициенты в кубическом сплайне, который интерполирует
    точки (x_1,y_1), (x_2, y_2),..., (x_n, y_n).
    Параметры:
        x: array_like, shape (n>2,).
           x-значение интерполируемых точек. Значения должны быть
           реальными и в строго возрастающем порядке.
        y: array_like, shape (n>2,)
           y-значение интерполируемых точек. Значения должны быть
           реальными
    Возвращается:
        array, shape (3, n). Коэффициенты b, c и d хранятся в
первой, второй и третьей строках соответственно.
    """
    x = np.asarray(x)
    y = np.asarray(y)
    n = len(x)
    dx = np.diff(x)
    dy = np.diff(y)

    # Задает вектор правой части уравнения
    rhs = np.zeros(n)
    rhs[1:-1] = 3*(dy[1:]/dx[1:] - dy[:-1]/dx[:-1])

    # Вычислит матрицу и сохранит упорядоченную по диагонали форму матрицы
    if (endpoint == "natural"):
        matrix = np.zeros((3, n))
        bands = (1, 1)
        matrix[1, 1:-1] = 2*(dx[:-1] + dx[1:]) # Диагональ
        matrix[1, 0] = matrix[1, -1] = 1
        matrix[0, 2:] = dx[1:] # Верхняя диагональ
        matrix[2, :-2] = dx[:-1] # Нижняя диагональ
    
    if (endpoint == "not-a-knot"):
        matrix = np.zeros((5, n))
        bands = (2, 2)
        matrix[2, 1:-1] = 2*(dx[:-1] + dx[1:]) # Диагональ
        matrix[1, 2:] = dx[1:] # Верхняя диагональ
        matrix[3, :-2] = dx[:-1] # Нижняя диагональ
        # First row
        matrix[2, 0] = dx[1]
        matrix[1, 1] = -dx[0] - dx[1]
        matrix[0, 2] = dx[0]
        # Last row
        matrix[2, -1] = dx[-2]
        matrix[3, -2] = -dx[-2] - dx[-3]
        matrix[4, -3] = dx[-1]
        
    # Вызовет решатель для полосчатой матрицы
    c = la.solve_banded(bands, matrix, rhs,
            overwrite_ab=True, overwrite_b=True, check_finite=False)
    
    # Найдет оставшиеся коэффициенты
    d = np.diff(c)/(3*dx)
    b = dy/dx - dx*(2*c[:-1] + c[1:])/3
    
    return b, c, d

Нам также нужна функция, которая может оценить сплайн с учетом коэффициентов.

In [ ]:
def cubic_spline_eval(x, xdata, ydata, b, c, d):
    """ Вычисляет кубический сплайн, который интерполирует {(xdata, ydata)}
    при x с коэффициентами b, c и d.
    Параметры:
        x:          array_like, shape(m,).
                    x-значения (ось), при которых вычисляется сплайн.
        a, b, c, d: array_like, shapes (n,), (n-1,), (n,) and (n-1,).
                    Коэффициенты сплайна.
    Возвращает:
        array, shape(m,). Оценка функции сплайна.
    """
    x = np.asarray(x)
    y = np.zeros(len(x))
    m = 0
    for i in range(len(xdata) - 1):
        n = np.sum(x < xdata[i + 1]) - m
        xx = x[m:m + n] - xdata[i]
        y[m:m + n] = ydata[i] + b[i]*xx + c[i]*xx**2 + d[i]*xx**3
        m = m + n
    xx = x[m:] - xdata[-2]
    y[m:] = ydata[-2] + b[-1]*xx + c[-2]*xx**2 + d[-1]*xx**3
    return y

## Пример
Теперь мы готовы найти кубический сплайн общего набора точек данных. Чтобы получить некоторую основу для сравнения, мы интерполируем некоторые точки из $\sin(x)$.

In [ ]:
def func(x):
    return np.sin(x)

xdata = np.asarray([0, 2, 4, 6, 8, 10])*np.pi/5
ydata = func(xdata)

x = np.linspace(xdata[0] - 2, xdata[-1] + 2, 200)
y = func(x)

b, c, d = cubic_spline_coeffs(xdata, ydata, "natural")
ya = cubic_spline_eval(x, xdata, ydata, b, c, d)

b, c, d = cubic_spline_coeffs(xdata, ydata, "not-a-knot")
yb = cubic_spline_eval(x, xdata, ydata, b, c, d)

plt.figure()
plt.plot(x, y, "--", label=r"$\sin(x)$")
plt.plot(x, ya, label="Natural cubic spline")
plt.plot(x, yb, label="Not-a-knot cubic spline")
plt.plot(xdata, ydata, 'o', label="Data points")
plt.xlim(x[0], x[-1])
plt.ylim(np.max(y)*1.1, np.min(y)*1.1)
plt.legend()
plt.show()

Обратите внимание, что мы определили кубические сплайны вне области точек данных. В неузловом кубическом сплайне внешние многочлены расширяются. То есть многочлен при $x<x_1$ такой же, как и при $x_1<x<x_2$. Естественный кубический сплайн, с другой стороны, определяет полином с "противоположной кривизной" вне точек данных. То есть, если сплайн изгибается от оси при $x_1<x<x_2$, он будет изгибаться к оси при $x<x_1$.

## Интерполяция параметризованных кривых
Также можно интерполировать параметризованные кривые с помощью сплайнов. Это делается путем выполнения интерполяции как по оси x, так и по оси y в зависимости от параметра кривой. Рассмотрим следующий пример:

In [ ]:
# Определит некоторые точки данных
xdata = [-0.5,-1.0,-0.5, 0.2, 1.5, 2.0, 1.0]
ydata = [ 5.0, 3.7, 1.0, 1.0,-0.5, 1.5, 4.0]
n = len(xdata)

# Значения параметра
t = np.linspace(0,1,100)

# Интерполяция кривой с использованием равномерно распределенных узлов параметров
ti = np.linspace(0,1,n)
# x-axis
b, c, d = cubic_spline_coeffs(ti, xdata, "not-a-knot")
Px = cubic_spline_eval(t, ti, xdata, b, c, d)
# y-axis
b, c, d = cubic_spline_coeffs(ti, ydata, "not-a-knot")
Py = cubic_spline_eval(t, ti, ydata, b, c, d)

plt.figure
plt.plot(Px,Py,'g',label='Uniformly disturbuted nodes.')
plt.plot(xdata,ydata,'r*',label='Data points.')
plt.title('Polynomial curve interpolation of a set data points.')
plt.legend(), plt.xlabel('x'), plt.ylabel('y')
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp;**Упражнение:** Используйте узлы Чебышева вместо узлов с равномерно распределенными параметрами. *Подсказка. См. [полиномиальная интерполяция](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/polynomial_interpolation.ipynb) notebook.*

## Почему Кубические сплайны?
Есть несколько преимуществ использования кубических сплайнов в отличие, например, от полиномиальной интерполяции высокого порядка или сплайнов более высокого порядка. В нашем блокноте по [полиномиальной интерполяции](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/polynomial_interpolation.ipynb) мы показали, что большие колебания (и, следовательно, большие ошибки) могут возникать при аппроксимации функции с использованием полинома высокого порядка. Это называется феноменом Рунге. Мы показали, что погрешность, обусловленная этим колебанием, минимизируется при использовании так называемых узлов Чебышева в качестве основы для интерполяции. Другим обходным путем является использование полиномиальных сплайнов низкого порядка.

Возможно, вам интересно, почему мы не используем квадратичный сплайн вместо кубических сплайнов. Аппроксимация функции с использованием большего количества оценок функций и полиномов более высокого порядка всегда должна быть лучше, верно? Неправильно! Конечно, мы можем утверждать, что полиномиальные сплайны более высокого порядка дают более плавные кривые, поскольку мы требуем, чтобы производные более высокого порядка были непрерывными. Однако это может привести к большим ошибкам из-за феномена Рунге. Кроме того, полиномиальные интерполяции более высокого порядка требуют большего количества точек данных. Таким образом, глобальное изменение кривой, вызванное изменением одной точки данных, увеличивается в полиномиальных сплайнах более высокого порядка.

## References and Further Reading

[1] Sauer, T.: Numerical Analysis international edition, second edition, Pearson 2014  
[2] Press, W.H., Teukolsky, S.A., Vetterling, W.T., Flannery, B.P.: Numerical Recipes, the Art of Scientific Computing, 3rd edition, Cambridge University Press 2007  

[scipy.interpolate](https://docs.scipy.org/doc/scipy-0.18.1/reference/interpolate.html) has several function for polynomial and spline interpolation, such as [CubicSpline](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.interpolate.CubicSpline.html#scipy.interpolate.CubicSpline) and [interp1d](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.interpolate.interp1d.html#scipy.interpolate.interp1d). Check them out!